# Function Calling with LLM APIs

## What This Covers

Function calling (also called "tool use") lets LLMs interact with external systems: APIs, databases, calculators, or any Python function you define. Instead of just generating text, the model can request function calls with specific parameters, you execute them, and return results.

This notebook shows:
- **OpenAI function calling**: Weather API example with explicit execution flow
- **Gemini function calling**: Same pattern using Google's API
- **Key pattern**: Model decides → You execute → Model generates final response

**The Core Idea:** You describe functions to the model. When appropriate, the model returns structured JSON saying "call this function with these arguments." You execute it and feed results back. The model then uses that information to respond to the user.

## What You'll Learn

- Define function schemas for OpenAI and Gemini
- Handle the full execution cycle (request → execute → respond)
- Understand when the model calls functions vs. responding directly
- Work with real APIs (weather data example)

**What's Next:** We'll explore LLM workflows where multiple LLMs coordinate.

**By the end:** You'll understand how to connect LLMs to external systems and handle the execution flow for both OpenAI and Gemini.

## OpenAI Function Calling

We'll start with OpenAI's function calling. The pattern:
1. Define a function (e.g., `get_weather`)
2. Describe it to the model using a schema
3. Call the model with the function available
4. Execute the function with the model's provided arguments
5. Send results back for the final response

Let's see it in action with a weather API.

**Step 1: Define the function.** We'll create `get_weather` that fetches temperature data from a free weather API:

In [1]:
import requests
from openai import OpenAI
from dotenv import load_dotenv
import json
import os


# Load environment variables from .env file
load_dotenv()

openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def get_weather(latitude, longitude):
    """Fetch current temperature from Open-Meteo API."""
    response = requests.get(
        f"https://api.open-meteo.com/v1/forecast?"
        f"latitude={latitude}&longitude={longitude}&current=temperature_2m"
    )
    data = response.json()
    return data['current']['temperature_2m']

**Step 2: Describe the function to the model.** We create a schema that tells OpenAI what the function does and what parameters it expects:

In [2]:
from openai import OpenAI
import json


tools = [{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current temperature for provided coordinates in celsius.",
        "parameters": {
            "type": "object",
            "properties": {
                "latitude": {"type": "number"},
                "longitude": {"type": "number"}
            },
            "required": ["latitude", "longitude"],
            "additionalProperties": False
        },
        "strict": True
    }
}]

**Step 3: Call the model.** Send a query to OpenAI with the function schema available. The model "decides" whether to call the function:

In [3]:
query = "What's the weather like in Sydney today?"

messages = [{"role": "user", "content": query }]

# Pass tools parameter so model can choose to call get_weather
completion = openai_client.chat.completions.create(
    model="gpt-5-mini",
    messages=messages,
    tools=tools,
)

completion.choices[0].message.tool_calls

[ChatCompletionMessageFunctionToolCall(id='call_HmlGdZuQs5i5mhqtbZJQdNLy', function=Function(arguments='{"latitude":-33.8688,"longitude":151.2093}', name='get_weather'), type='function')]

**Step 4: Execute the function.** Extract the function call from the model's response and run it with the provided arguments:

In [4]:
# Extract the function name and arguments from model's response
tool_call = completion.choices[0].message.tool_calls[0]
args = json.loads(tool_call.function.arguments)

# Execute the actual function
result = get_weather(args["latitude"], args["longitude"])

**Step 5: Send results back.** Add the function result to the conversation and call the model again for the final user-facing response:

In [5]:
# Add both the model's tool call and the result to conversation history
messages.append(completion.choices[0].message)  
messages.append({
    "role": "tool",
    "tool_call_id": tool_call.id,
    "content": str(result)
})

# Call model again with the function result available
completion_2 = openai_client.chat.completions.create(
    model="gpt-5-mini",
    messages=messages,
    tools=tools,
)

In [6]:
completion_2.choices[0].message.content

'Current temperature in Sydney (CBD) is about 25.5°C (≈78.0°F) right now — pleasantly warm. \n\nWould you like a full forecast (hourly/daily), precipitation chance, wind/humidity, or clothing suggestions?'

## Gemini Function Calling

The same pattern works with Gemini's API, just with different syntax:
1. Define and configure the function as a tool
2. Call the model with the tool available
3. Execute the function and send results back for the final response

We'll use the same `get_weather` function and query defined above.

First, import Gemini and create the client:

In [7]:
from google import genai
from google.genai import types

gem_client = genai.Client(api_key=os.getenv("GOOGLE_API_KEY"))

**Step 1: Define and configure the tool.** Create the function schema and wrap it as a Gemini tool:

In [8]:
# Using the same get_weather function defined above
get_weather_tool = {
    "name": "get_weather",
    "description": "Get current temperature for provided coordinates in celsius. When the user asks about weather in a city you should determine the latitude and longitude coordinates for that city.",
    "parameters": {
        "type": "object",
        "properties": {
            "latitude": {"type": "number"},
            "longitude": {"type": "number"}
        },
        "required": ["latitude", "longitude"]
    }
}

tools = types.Tool(function_declarations=[get_weather_tool])
config = types.GenerateContentConfig(tools=[tools])

**Step 2: Call the model.** Send the query with tools available:

In [11]:
query = "What's the weather like in Sydney today?"

contents = [
    types.Content(role="user", parts=[types.Part(text=query)])
]

response = gem_client.models.generate_content(
    model="gemini-2.5-flash",
    contents=contents,
    config=config,
)

print(response.candidates[0].content.parts[0].function_call)

id=None args={'latitude': -33.8688, 'longitude': 151.2093} name='get_weather' partial_args=None will_continue=None


**Step 3: Execute and respond.** Extract the function call, run it, and send results back for the final response:

In [12]:
# Check if Gemini called the function
if response.candidates[0].content.parts[0].function_call:
    tool_call = response.candidates[0].content.parts[0].function_call
    
    # Execute the actual function with Gemini's provided arguments
    result = get_weather(**tool_call.args)
    
    # Format result for Gemini
    function_response_part = types.Part.from_function_response(
        name=tool_call.name,
        response={"result": result}
    )
    
    # Add function call and result to conversation
    contents.append(response.candidates[0].content)
    contents.append(types.Content(role="user", parts=[function_response_part]))
    
    # Call model again with function result
    final_response = gem_client.models.generate_content(
        model="gemini-2.5-flash",
        contents=contents,
        config=config,
    )
    
    print(final_response.text)
else:
    # Model responded directly without calling function
    print("No function call - direct response:")
    print(response.text)

The weather in Sydney is 25.5 degrees Celsius.


## Enriching Data with Search

In [14]:
from tavily import TavilyClient
from openai import OpenAI
import os
import json

# Initialize clients
tavily_client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))

name = "Hugo Bowne-Anderson"
print(f"Searching for {name}...")

try:
    # 1. GET DATA (Tavily)
    query = f"{name} AI machine learning data science personal website" 
    response = tavily_client.search(query, max_results=10)
    
    # Format results
    search_context = []
    print(f"Tavily found {len(response.get('results', []))} results:")
    for idx, result in enumerate(response.get("results", []), 1):
        url = result.get('url', '')
        title = result.get('title', '')
        print(f"{idx}. {url} - {title}")
        search_context.append(
            f"URL: {url}\n"
            f"Title: {title}\n"
            f"Content: {result.get('content', '')[:300]}\n"
            "---"
        )
    full_context = "\n".join(search_context)

    # 2. EVALUATE (OpenAI GPT-4o-mini)
    prompt = f"""
    I am looking for the OFFICIAL personal website for: {name}

    IMPORTANT CONTEXT: This person works in Data Science, AI, or Machine Learning. 

    If a website belongs to someone with the same name but a different profession (e.g., a lawyer or doctor), REJECT IT.

    Here are the search results:
    {full_context}

    Task:
    1. Analyze the results.
    2. Identify the official personal website (Priority: Personal Domain > Academic Profile > LinkedIn/Twitter).
    3. Verify they are in the AI/ML/Data field.
    4. If no valid website is found, return null.

    Output JSON format: {{ "url": "https://..." or null }}
    """

    llm_response = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        response_format={"type": "json_object"}
    )
    
    # 3. PARSE
    result_json = json.loads(llm_response.choices[0].message.content)
    found_url = result_json.get("url")

    if found_url:
        print(f"Found: {found_url}")
    else:
        print(f"No good match found.")
        
except Exception as e:
    print(f"Error processing {name}: {e}")

Searching for Hugo Bowne-Anderson...
Tavily found 10 results:
1. https://hugobowne.github.io/hugo-blog/ - Hugo's blog
2. https://x.com/hugobowne?lang=en - Hugo Bowne-Anderson (@hugobowne) / Posts / X
3. https://au.linkedin.com/in/hugo-bowne-anderson-045939a5 - Hugo Bowne-Anderson - Data and AI scientist, consultant. ...
4. https://hugobowne.github.io/ - hugo bowne-anderson - data scientist
5. https://www.turnthelenspodcast.com/episode/hugo-bowne-anderson-data-scientists-evangelists-easy-button-turn-the-lens-15 - Hugo Bowne-Anderson: Data Scientists, Evangelists, Easy ...
6. https://www.youtube.com/watch?v=eC3RNuI6ow0 - How to Build and Evaluate AI systems in the Age of LLMs ...
7. https://vanishinggradients.fireside.fm/hosts/hugobowne - Hugo Bowne-Anderson
8. https://learnbayesstats.com/episode/122-learning-and-teaching-in-the-age-of-ai-hugo-bowne-anderson - 122 Learning And Teaching In The Age Of Ai Hugo Bowne ...
9. https://hugobowne.substack.com/p/rethinking-data-science-ml-and-ai -

## Summary

You've now seen function calling with both OpenAI and Gemini. The core pattern is the same across providers:
- Define functions and describe them to the model
- Model decides when to call them and with what arguments
- You execute the actual function
- Results go back to the model for a final response

**Key insight:** The model doesn't execute functions: it just requests them. You control what actually runs.

**What's next:** Session 6b explores multi-LLM workflows where multiple models coordinate to accomplish complex tasks.